In [1]:
import pandas as pd
import json
import glob, os
import fileinput
import re
from langdetect import detect

In [2]:
#read files
#data_dir = os.path.dirname(os.path.realpath(__file__))
data_dir = "data" #os.getcwd()
timelines_dir = "%s/timelines" % data_dir
test_dir = "%s/test1" % timelines_dir

#data = open("data/timelines/test1/63299591","r")
#data = open("data/timelines/test1/3578166252","r")
#data = "data/timelines/test1/3578166252"
#data_file = "%s/133684052" % test_dir

In [3]:
list_of_files = [os.path.join(dirs, file) for dirs in [os.path.join(timelines_dir, filename) for filename in os.listdir(timelines_dir) if not filename.endswith('.json')] for file in os.listdir(dirs)]


In [33]:
test_files = [os.path.join(test_dir, file) for file in os.listdir(test_dir)]

In [17]:
def clean_text(text):
    text = text.replace("\n"," ")
    text = text.replace("\r"," ")
    text = re.sub('@[^\s]+','',text)       # removes all the usernames
    text = re.sub(r"http\S+", "", text)    # removes all the links
    text = re.sub(r"RT ", "", text)        # removes all the retweet-prefixes
    return text

In [106]:
def preprocess_files_to_dataframe(files):
    # this function does a cuple of things at a time
    # it takes all files (don't forget to iterate through all the directories) and preprocess them
    # preprocessing means the selection of columns, the cleaning of the text from links and such,
    # get the lenghts of each text, deleting the short and redundand ones
    
    tweet_df = pd.DataFrame()
    for file in files:
        with open(file, 'r') as tweet:
            aux_df = pd.read_json(tweet, orient = 'values', lines=True)
            tweet_df = pd.concat([tweet_df, aux_df], ignore_index=True, sort=False)
            #print(aux_df.head())
            #print(tweet_df.head())
            tweet.close()
    tweet_df = tweet_df.drop(columns=['contributors', 'coordinates',
       'entities', 'extended_entities', 'favorite_count', 'favorited', 
       'geo', 'id', 'id_str', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status',
       'place', 'possibly_sensitive', 'retweet_count', 'retweeted',
       'source', 'truncated', 'user','quoted_status', 'quoted_status_id', 'quoted_status_id_str',
       'retweeted_status'])
    tweet_df.reset_index(inplace=True) # create index enumeration
    tweet_df['clean_text'] = tweet_df.full_text.apply(lambda x: clean_text(x)) # clean text from unnecessary parts
    tweet_df['len_cleaned_text'] = tweet_df.clean_text.apply(lambda x: len(x)) 
    tweet_df['len_full_text'] = tweet_df.full_text.apply(lambda x: len(x))
    print("first drop before lenghts-reduction: %i" % len(tweet_df['len_cleaned_text'].index))
    tweet_df.drop(tweet_df[tweet_df['len_cleaned_text'] < 5].index, axis=0, inplace=True) # delete all raws with short texts
    print("second drop before clean_text-reduction: %i" % len(tweet_df['len_cleaned_text'].index))
    tweet_df.drop_duplicates(subset=['clean_text'], keep='first', inplace=True) # delete redundand entries
    print("third drop before lang_detection: %i" % len(tweet_df['len_cleaned_text'].index))
    tweet_df['detected_lang'] = tweet_df.clean_text.apply(lambda x: detect(x)) 
#    tweet_df.drop(tweet_df[tweet_df['clean_text'].map(detect) != 'ja' ].index ,axis=0, inplace=True) # delete anything which is recognized as non-jap 
    print("last drop after lang_detection: %i" % len(tweet_df['len_cleaned_text'].index))
    return tweet_df

In [107]:
%time tweet_df = preprocess_files_to_dataframe(test_files)

first drop before lenghts-reduction: 3253
second drop before clean_text-reduction: 3252
third drop before lang_detection: 145
last drop after lang_detection: 145
CPU times: user 2.95 s, sys: 212 ms, total: 3.16 s
Wall time: 3.22 s


In [109]:
tweet_df.head(1000)

,index,created_at,display_text_range,full_text,lang,clean_text,len_cleaned_text,len_full_text,detected_lang
0,0,2018-03-30 04:07:32,"[0, 114]",辛味噌つけ麺辛さMAX\n\n辛いのがー食べたい！\nで近所ですぐ手に入るつけ麺屋さん。\n...,ja,辛味噌つけ麺辛さMAX 辛いのがー食べたい！ で近所ですぐ手に入るつけ麺屋さん。 写真撮...,91,114,ja
1,1,2018-03-30 04:30:10,"[0, 65]",でっかい外ロケがしばらくないから、いまだ！と親知らず抜きました。どーなるかなー。腫れませんよ...,ja,でっかい外ロケがしばらくないから、いまだ！と親知らず抜きました。どーなるかなー。腫れませんよ...,65,65,ja
2,2,2018-03-30 08:50:44,"[0, 96]",#おなかがすいたらモンスター 登録者さん28万人・・・・！！！本当に皆楽しみにしてくれてあり...,ja,#おなかがすいたらモンスター 登録者さん28万人・・・・！！！本当に皆楽しみにしてくれてあり...,97,120,ja
3,3,2018-03-30 11:35:32,"[0, 85]",今日の #おなかがすいたらモンスター はタマゴ30個つかった #ミルフィーユ #オムレツ V...,ja,今日の #おなかがすいたらモンスター はタマゴ30個つかった #ミルフィーユ #オムレツ V...,62,85,ja
5,5,2018-03-27 15:22:26,"[13, 65]",@mango_mango あなたが選んだ秋山さんはコチラ↓\n \n他の動画も見てね！\nh...,ja,あなたが選んだ秋山さんはコチラ↓ 他の動画も見てね！,31,89,ja
14,14,2018-03-27 13:00:00,"[0, 111]",＿人人人人人人人人人＿\n＞ マイル山分け🎉 ＜\n￣Y^Y^Y^Y^Y^Y^Y^Y^￣\n...,ja,＿人人人人人人人人人＿ ＞ マイル山分け🎉 ＜ ￣Y^Y^Y^Y^Y^Y^Y^Y^￣ #プ...,88,111,ja
15,15,2018-03-27 12:58:06,"[11, 117]",@565131091 あなたが選んだ #新・竜兵会 の掟はこちら！\n／\n掟その1「カンパ...,ja,あなたが選んだ #新・竜兵会 の掟はこちら！ ／ 掟その1「カンパイはなまウーで」 ＼ ...,85,141,ja
16,16,2018-03-27 12:37:49,"[13, 120]",@0804yokopin あなたが選んだ #新・竜兵会 の掟はこちら！\n／\n掟その2「烏...,ja,あなたが選んだ #新・竜兵会 の掟はこちら！ ／ 掟その2「烏龍茶で食事を楽しむべし」 ＼...,86,144,ja
21,21,2018-03-27 11:00:00,"[0, 166]",#香取慎吾 さん放送楽しみにしています。\n「サントリー オールフリー」はこれからも香取さん...,ja,#香取慎吾 さん放送楽しみにしています。 「サントリー オールフリー」はこれからも香取さんの...,143,166,ja
25,25,2018-03-27 08:50:31,"[10, 56]",@jyapp417 気が付いたらクラフトボスを箱買い・・だなんてΣヾ(＞▽＜)ﾉ💕 お楽しみ...,ja,気が付いたらクラフトボスを箱買い・・だなんてΣヾ(＞▽＜)ﾉ💕 お楽しみいただけますように🎵,47,56,ja


In [ ]:
# missing lang check: done
# tokenization

In [22]:
for index in range(100):
    print(30*'_')
    if detect(tweet_df['clean_text'].iloc[index]) == "ja":
        print("cool. here's the tweet: %s" % tweet_df['clean_text'].iloc[index])
    else:
        print("nogo bro. here's the text anyway: \n\n %s. \n Guess it's %s" % (tweet_df['clean_text'].iloc[index],detect(tweet_df['clean_text'].iloc[index])))

______________________________
cool. here's the tweet: 昨日の動画は、 #チーズタッカルビ だったよーん❤  うひょおおおおおー！！！！ プロフィールのYouTubeリンクから見てね☺️ #おなかがすいたらモンスター #おなもん… 
______________________________
cool. here's the tweet:  からかったですねー！フルーティな香りに騙されるとかれぇえええってなりますww
______________________________
cool. here's the tweet: 今日の動画は、広島のもみじまんじゅう食べ比べー！ 後半美味しすぎてまじ困った。 48個じゃ全然足りないやつ！ #ロシアン佐藤 #おなかがすいたらモンスター  
______________________________
cool. here's the tweet: ひょーいとひょーいとひょひょいと日本！  ひろしまー！もみじまんじゅう！  日曜の動画はもみじまんじゅう！  プロフィールのYouTubeリンクから見てね☺️ #おなかがすいたらモンスター… 
______________________________
cool. here's the tweet:  あなたが選んだ #新・竜兵会 の掟はこちら！ ／ 掟その4「 #会の〆は烏龍茶で息スッキリ」 ＼  その他の掟動画は↓からチェックしてね！ LINEスタンプもプレゼント中！  
______________________________
cool. here's the tweet:  あなたが選んだ #新・竜兵会 の掟はこちら！ ／ 掟その1「カンパイはなまウーで」  ＼  その他の掟動画は↓からチェックしてね！ LINEスタンプもプレゼント中！  
______________________________
cool. here's the tweet:  あなたが選んだ #新・竜兵会 の掟はこちら！ ／ 掟その2「烏龍茶で食事を楽しむべし」 ＼  その他の掟動画は↓からチェックしてね！ LINEスタンプもプレゼント中！  
______________________________
cool. here's th

nogo bro. here's the text anyway: 

  There is no remedy for love but to love more. Henry David Thoreau. 
 Guess it's en
______________________________
nogo bro. here's the text anyway: 

  Beck / Up All Night (2017)   Follow Kenta's  on twitter!. 
 Guess it's en
______________________________
cool. here's the tweet:  『アマル・ハヤーティ』(1965年／昭和40年)  نادر : حفلة أمل حياتى لأم كلثوم كامله  アブドゥル＝ワッハーブとの共作第三弾。前説まで完全収録で超長…
______________________________
cool. here's the tweet:  本当に腸内環境整備を考えるなら、食べたり飲んだりするヨーグルトより、乳酸菌サプリのほうが効きます。
______________________________
cool. here's the tweet: 「返還がこんなに大変と知っていたら…」　奨学金の苦悩：朝日新聞デジタル 
______________________________
cool. here's the tweet: 民意が首相を見限る条件とは　支持率１桁政権から考える：朝日新聞デジタル 
______________________________
cool. here's the tweet: 行き先はＦＢでランダムに　住む場所も無作為に選ぶ人生：朝日新聞デジタル 
______________________________
cool. here's the tweet:  グースエッグシアターアメブロ更新しました    #グースエッグシアター #アメブロ更新


In [ ]:
# check whether tweet is japanese
            if detect(tweet) != "ja":
                #lang = detect(tweet)
                #print(lang)
                print(tweet)
                continue

In [6]:
def entry_check(tweet,collection_data):
    # In order to reduce redundancy, each tweet will be checked, such that doubles will be thrown out
    
    # tweet is a variable for the full_text in the raw_file
    # collection_data is the file with the manipulated full_text for collection
    rawfile = open(collection_data,'r')
    for lines in rawfile:
        #print(lines)
        if tweet+"\n" == lines:
            return True

In [7]:
entry_check(files[0])

TypeError: entry_check() missing 1 required positional argument: 'collection_data'

In [3]:
def raw_data_collect(rawdata,collection):
    with open(rawdata,"r") as data:
        for lines in data:

            #ToDo
            #Remove all links, @usernames

            # remove empty spaces and unwanted line breaks
            json_lines = json.loads(lines)
            tweet = json_lines["full_text"]
            tweet = tweet.replace("\n","")
            tweet = tweet.replace("\r","")
            tweet = re.sub('@[^\s]+','',tweet)       # removes all the usernames
            tweet = re.sub(r"http\S+", "", tweet)    # removes all the links

            # check whether tweet is japanese
            if detect(tweet) != "ja":
                #lang = detect(tweet)
                #print(lang)
                print(tweet)
                continue

            # exclude sentences shorter than 5 characters
            elif len(tweet) < 5:
                print("deleted: ",tweet)
                continue
            else:
                f = open(collection,"a")
                if entry_check(tweet,collection) == True:
                    continue
                f.write(tweet+"\n")
                f.close()

In [246]:
list_of_files = [os.path.join(dirs, file) for dirs in [os.path.join(timelines_dir, filename) for filename in os.listdir(timelines_dir) if not filename.endswith('.json')] for file in os.listdir(dirs)]
for path in list_of_files:
    print(path)

data/timelines/20180401/3578166252
data/timelines/20180401/133684052
data/timelines/20180401/63299591
data/timelines/20180401/93196638
data/timelines/20180403/3578166252
data/timelines/20180403/133684052
data/timelines/20180403/63299591
data/timelines/20180403/93196638
data/timelines/20180402/3578166252
data/timelines/20180402/133684052
data/timelines/20180402/63299591
data/timelines/20180402/93196638
data/timelines/20180330/3578166252
data/timelines/20180330/133684052
data/timelines/20180330/63299591
data/timelines/20180330/93196638
data/timelines/20180329/3578166252
data/timelines/20180329/133684052
data/timelines/20180329/63299591
data/timelines/20180329/93196638
data/timelines/20180331/3578166252
data/timelines/20180331/133684052
data/timelines/20180331/63299591
data/timelines/20180331/93196638
data/timelines/20180328/3578166252
data/timelines/test1/3578166252
data/timelines/test1/133684052
data/timelines/test1/63299591
data/timelines/20180327/133684052
data/timelines/20180327/6329

In [4]:
def file_iterator(aim_file):
    #todo_path = os.path.join(os.getcwd(), "data", "timelines")
    #todo_path_current = os.getcwd()

    #ROOT = [os.path.join(todo_path, filename) for filename in filter(os.path.isdir, os.listdir(todo_path))]
    todo_path = timelines_dir
    ROOT = [os.path.join(todo_path, filename) for filename in os.listdir(todo_path)]


    for x in ROOT:
        if os.path.isdir(x):
            for item in os.listdir(x):
                print(item)
                data = os.path.join(x, item)
                #print(data)
                raw_data_collect(data,aim_file)

In [7]:
file_iterator("raw2")

3578166252
133684052
63299591
93196638
豆乳を投入(=ﾟωﾟ)ﾉ。RT  豆乳を投入(=ﾟωﾟ)ﾉ　　#豆乳を投入
RT  HELLO, DOLLY !The NEW Broadway Cast Recordingはてなブログに投稿しました #はてなブログRCA Records ANL1-2849 - ひとりDiscogs
RT  Cyril Mokaiesh &amp; Bernard Lavilliers - La loi du marché (2016)  新自由主義者に好き放題をされている現代世界について歌う「市場の法則」…
RT  はてなブログに投稿しました #はてなブログ1938.02.17. SLIM and SLAM [SLIM GAILLARD 2nd session] - ...years ago today
RT  Sonny Clay's Plantation Orchestra 
RT  King Oliver's Band 1931 
RT  Bennie Moten's Radio Orchestra 
RT  Buffalo SpringfieldのMr. Soul  #NowPlaying
RT  Daddy Dewdrop / Chick-A-Boom (Don't Ya Jes' Love It) (1971 US #9) Kenta's 
RT  Saturday Night Flow - yeah LAST Saturday night! Tonight I’m stuck in the crib helping a certain college student with 5 P…
RT  . celebrates the legacy of Blue Note Records from our founding in 1939 to the vibrant sounds of today with the…
RT  😢😰 
Diz and Bird. left, Max.  
RT  There is no remedy for love but to love more. Henry David Thoreau
RT  Beck / Up All Night (2017) Kent

LangDetectException: No features in text.